After ATOMICA was trained for 3 epoch on predicting made up affinities, here we predict more affinities.

In [1]:
import numpy as np
import json
import os
import pandas as pd
import sys
import torch
from tqdm import tqdm

current_directory = os.getcwd()
parent_directory = os.path.dirname(os.path.dirname(current_directory))
sys.path.insert(0, parent_directory)

from data.dataset import PDBDataset
from models.affinity_predictor import AffinityPredictor
from trainers.abs_trainer import Trainer

In [2]:
model = AffinityPredictor.load_from_config_and_weights(
    "/home/sascha/data/Projects/affinity_project/affinity/src/submodules/ATOMICA/trial/process_complexes/trial_checkpoints/version_2/checkpoint/config.json",
    "/home/sascha/data/Projects/affinity_project/affinity/src/submodules/ATOMICA/trial/process_complexes/trial_checkpoints/version_2/checkpoint/epoch2_step801.pt"
)

/home/sascha/data/Projects/affinity_project/affinity/src/submodules/ATOMICA/atomicaenv/lib/python3.9/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/sascha/data/Projects/affinity_project/affinity/src/submodules/ATOMICA/atomicaenv/lib/python3.9/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/sascha/data/Projects/affinity_project/affinity/src/submodules/ATOMICA/atomicaenv/lib/python3.9/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/sascha/data/Projects/affinity_project/affinity/src/submodules/ATOMICA/atomicaenv/lib/python3.9/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
/home/sascha/data/Projects

In [3]:
print(f"{sum(p.numel() for p in model.parameters())} parameters in total")
print(f"{sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters in total")

7458017 parameters in total
7439809 trainable parameters in total


In [4]:
dataset = PDBDataset("/home/sascha/data/Projects/affinity_project/affinity/src/submodules/ATOMICA/trial/process_complexes/test_labelled_processed_pdbs.pkl")

In [5]:
import csv
batch_size = 1  # Adjust batch size as needed

output_path = "affinity_predictions.csv"
with open(output_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "predicted_affinity"])

    for i in range(0, len(dataset), batch_size):
        batch = PDBDataset.collate_fn([dataset[j] for j in range(i, min(i + batch_size, len(dataset)))])
        batch = Trainer.to_device(batch, 'cpu')
        if 'label' not in batch:
            with torch.no_grad():
                dummy_pred = model.infer({**batch, 'label': torch.zeros(1)})
            label_shape = dummy_pred[1].shape if isinstance(dummy_pred, tuple) else dummy_pred.shape
            batch['label'] = torch.zeros(label_shape)

        with torch.no_grad():
            prediction = model.infer(batch)
        # Get IDs and predictions for the whole batch
        pdb_ids = batch.get('id', [f"sample_{i+j}" for j in range(len(batch['label']))])
        pred_values = prediction[1] if isinstance(prediction, tuple) else prediction
        pred_values = pred_values.cpu().numpy().flatten()
        for pdb_id, pred_value in zip(pdb_ids, pred_values):
            writer.writerow([pdb_id, pred_value])

        del batch, prediction
        torch.cuda.empty_cache()

print(f"Predictions saved to {output_path}")

Predictions saved to affinity_predictions.csv


In [6]:
# After creating the batch and moving to device
if 'label' not in batch:
    # Run a dummy forward pass to get the output shape
    with torch.no_grad():
        dummy_pred = model.infer({**batch, 'label': torch.zeros(1)})
    label_shape = dummy_pred[1].shape if isinstance(dummy_pred, tuple) else dummy_pred.shape
    batch['label'] = torch.zeros(label_shape)

NameError: name 'batch' is not defined

In [ ]:
batch.keys()